<a href="https://colab.research.google.com/github/anmolag190153/BCS_summer_project_SER/blob/main/Speech_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPEECH EMOTION RECOGNITION PROJECT
## BCS-IITK


# Emotions Classification

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import CSVLogger
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# path1='/content/drive/My Drive/Ravdess/Actor_01/03-01-08-02-02-02-01.wav'
# ipd.Audio(path1)

In [10]:
# actor_1,sr=librosa.load(path1)
# print(actor_1.shape)
# print(sr)
sr=22100

In [7]:
# plt.figure(figsize=(15,5))
# librosa.display.waveplot(actor_1,sr=sr)
# plt.ylim((-0.1,0.1))
# plt.show()

In [7]:
def build_fc_model(input_shape=(181,1), num_class=8):
    
    
    model=tf.keras.models.Sequential([tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Conv1D(256,3,input_shape=input_shape,activation='relu'),
                                    tf.keras.layers.MaxPool1D(2),
                                    tf.keras.layers.Conv1D(256,3,activation='relu'),
                                    tf.keras.layers.MaxPool1D(2),
                                    tf.keras.layers.Conv1D(256,3,activation='relu'),
                                    tf.keras.layers.MaxPool1D(2),
                                    # tf.keras.layers.Dense(512,activation='relu',input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(1e-2)),
                                    # tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
                                    # tf.keras.layers.Dropout(0.1),
                                    # tf.keras.layers.Dense(128,activation='relu'),
                                    # tf.keras.layers.Dense(64,activation='relu'),
                                    tf.keras.layers.Dense(num_class,activation='softmax')
                                    ])

    return model

In [8]:
def extract_feature(file):
    result=np.array([])
    mfcc=np.mean(librosa.feature.mfcc(file,sr=sr,n_mfcc=40).T,axis=0)
    result=np.hstack((result,mfcc))
    chroma=np.mean(librosa.feature.chroma_stft(file,sr=sr).T,axis=0)
    result=np.hstack((result,chroma))
    mel=np.mean(librosa.feature.melspectrogram(file,sr=sr).T,axis=0)
    result=np.hstack((result,mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr=sr).T,axis=0)
    result=np.hstack((result,spectral_rolloff))
    return result

# Loading Data
We are using the RAVDESS dataset for this project.

In [4]:
path='Audio Datasets'
k=0
x_data=np.ndarray(shape=(24*60,120000))
y_data=np.ndarray(shape=(24*60,1))
for i,actor in enumerate(os.listdir(path)):
    path1=os.path.join(path,actor)
    for j,voice in enumerate(os.listdir(path1)):
        if voice=='test':
            continue
        b=voice[6:8]
        y_data[k,0]=b
        y_data[k,0]=y_data[k,0]-1
        temp_path=os.path.join(path1,voice)
        # print(temp_path)
        var,_=librosa.load(temp_path)
        x_data[k,:var.shape[0]]=var[:]
        x_data[k,var.shape[0]:]=0
        k=k+1

# Extracting Features
We have used mfccs, chroma frequency, mel spectogram and spectral centroid as audio features in this project.

In [11]:
x=[]
for i in range(24*60):
    feature=extract_feature(x_data[i,:])
    x.append(feature)

**Scaling** the data and splitting data for **training** and **validation**

In [19]:
x_train, x_test,y_train,y_test = train_test_split(np.array(x), y_data, test_size=0.2, random_state=9)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
print((x_train.shape,x_test.shape,y_train.shape))
x_train = x_train.reshape(1152,181,1)
x_test = x_test.reshape(288,181,1)

((1152, 181), (288, 181), (1152, 1))


# Training the model
The model classifies human voices in 8 different classes of emotions

**label**   **emotion**

0   =   neutral

1   =   calm

2   =   happy

3   =   sad

4   =   angry

5   =   fearful

6   =   disgust

7   =   surprised

In [23]:
model=build_fc_model()
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# csv_logger = CSVLogger('model_history.log', separator=',', append=False)
history=model.fit(x_train,y_train,epochs=200,batch_size=256,validation_data=(x_test,y_test))

l_accuracy: 0.6285
Epoch 61/200
5/5 [==============================] - 1s 191ms/step - loss: 0.3068 - accuracy: 0.9731 - val_loss: 2.9596 - val_accuracy: 0.6354
Epoch 62/200
5/5 [==============================] - 1s 207ms/step - loss: 0.3180 - accuracy: 0.9661 - val_loss: 3.2376 - val_accuracy: 0.5729
Epoch 63/200
5/5 [==============================] - 1s 187ms/step - loss: 0.3328 - accuracy: 0.9540 - val_loss: 3.3161 - val_accuracy: 0.5972
Epoch 64/200
5/5 [==============================] - 1s 186ms/step - loss: 0.2917 - accuracy: 0.9792 - val_loss: 3.2755 - val_accuracy: 0.6076
Epoch 65/200
5/5 [==============================] - 1s 187ms/step - loss: 0.3011 - accuracy: 0.9696 - val_loss: 3.3647 - val_accuracy: 0.5938
Epoch 66/200
5/5 [==============================] - 1s 188ms/step - loss: 0.2950 - accuracy: 0.9679 - val_loss: 3.7043 - val_accuracy: 0.6146
Epoch 67/200
5/5 [==============================] - 1s 241ms/step - loss: 0.2812 - accuracy: 0.9731 - val_loss: 3.9932 - val_accu

In [31]:
# model.save('mlp_model_Team-4')

INFO:tensorflow:Assets written to: mlp_model_Team-4/assets


In [24]:
result = model.evaluate(x_test,y_test)
print('Accuracy obtained on validation is ',result[1]*100,'%')

9/9 [==============================] - 0s 11ms/step - loss: 3.2970 - accuracy: 0.6389
Accuracy obtained on validation is  63.88888955116272 %
